In [1]:
!wget --no-check http://d28rh4a8wq0iu5.cloudfront.net/ads1/code/bm_preproc.py
!wget --no-check http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta

--2021-05-25 00:10:50--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/code/bm_preproc.py
Resolving d28rh4a8wq0iu5.cloudfront.net... 13.226.211.37, 13.226.211.214, 13.226.211.135, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net|13.226.211.37|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9400 (9.2K) [application/octet-stream]
Saving to: 'bm_preproc.py'

bm_preproc.py       100%[===================>]   9.18K  --.-KB/s    in 0s      

2021-05-25 00:10:50 (25.7 MB/s) - 'bm_preproc.py' saved [9400/9400]

--2021-05-25 00:10:51--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
Resolving d28rh4a8wq0iu5.cloudfront.net... 13.226.211.214, 13.226.211.135, 13.226.211.176, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net|13.226.211.214|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810105 (791K) [application/octet-stream]
Saving to: 'chr1.GRCh38.excerpt.fasta'

chr1.GRCh38.excerpt 100%[===================>] 791.12K  1.81MB/s  

In [18]:
import bm_preproc as p_bm
from bm_preproc import BoyerMoore

In [17]:
def boyer_moore_with_counts(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    char_count = 0
    align_count = 0
    while i < len(t) - len(p) + 1:
        align_count += 1
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            char_count += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences, align_count, char_count

In [11]:
def naive_with_counts(p, t):
    occurrences = []
    char_count = 0
    align_count = 0
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        align_count += 1
        match = True
        for j in range(len(p)):  # loop over characters
            char_count += 1
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences, align_count, char_count

In [21]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [189]:
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
t = readGenome('chr1.GRCh38.excerpt.fasta')
p_bm = BoyerMoore(p, 'AGCT')
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[56922] 799954 984143


In [25]:
!wget --no-check https://d28rh4a8wq0iu5.cloudfront.net/ads1/code/kmer_index.py

--2021-05-25 00:42:54--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/code/kmer_index.py
Resolving d28rh4a8wq0iu5.cloudfront.net... 13.226.211.135, 13.226.211.176, 13.226.211.37, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net|13.226.211.135|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 971 [application/octet-stream]
Saving to: 'kmer_index.py'

kmer_index.py       100%[===================>]     971  --.-KB/s    in 0s      

2021-05-25 00:42:55 (66.1 MB/s) - 'kmer_index.py' saved [971/971]



In [81]:
import bisect
class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer
    
    def query(self, p):
        ''' Return index hits for first k-mer of P '''
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [190]:
def approximate_match(p, t, n):
    seg_len = int(round(len(p) / (n+1)))
    all_matches = set()
    index_hits = 0
    for i in range(n+1):
        start = i*seg_len
        end = min((i+1)*seg_len, len(p))
        index = Index(t, 8)
        matches = index.query(p[start:end])
        index_hits += len(matches)
        for m in matches:
            if (m - start) < 0 or m-start+len(p) > len(t):
                continue            
            mismatches = 0
            for j in range(0,start):
                if p[j] != t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for k in range(end,len(p)):
                if p[k] != t[m-start+k]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(m-start)
    return list(all_matches), index_hits

In [185]:
def naive_2mm(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        mm = 0
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                mm += 1
                if mm > 2:
                    match = False
                    break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences

In [191]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
approximate_match(p, t, 2)[1]

90

In [192]:
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
        
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [198]:
def query_subseq(p, t, subseq_ind):
    segment_length = int(round(len(p) / subseq_ind.ival))
    all_matches = set()
    idx_hits = 0
    for i in range(subseq_ind.ival):
        start = i
        matches = subseq_ind.query(p[start:])
        idx_hits += len(matches)
        # Extend matching segments to see if whole p matches
        for m in matches:
            if m < start or m - start + len(p) > len(t):
                continue

            mismatches = 0
            for j in range(0, len(p)):
                if not p[j] == t[m - start + j]:
                    mismatches += 1
                    if mismatches >= subseq_ind.ival:
                        break

            if mismatches < subseq_ind.ival:
                all_matches.add(m - start)
    return list(all_matches), idx_hits

In [199]:
ind = SubseqIndex(t, 8, 3)

In [201]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
query_subseq(p, t, ind)[1]

79